In [1]:
'''
Author information:
Joey R. Muffoletto
University of Texas at Austin
Autonomous Systems Group
jrmuff@utexas.edu
'''


'\nAuthor information:\nJoey R. Muffoletto\nUniversity of Texas at Austin\nAutonomous Systems Group\njrmuff@utexas.edu\n'

In [2]:
'''
imports
'''
import sys
import os
sys.path.insert(0, os.path.abspath('C:\\Users\\Joey\\PycharmProjects\\Automata-Testing'))
import copy
import optimization_functions
import graph_manager
import graph_test_methods
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
os.environ["PATH"] += os.pathsep + 'C:/Program Files/Graphviz 2.44.1/bin'

c:\users\joey\appdata\local\programs\python\python37\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
c:\users\joey\appdata\local\programs\python\python37\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
c:\users\joey\appdata\local\programs\python\python37\lib\site-packages\numpy\.libs\libopenblas.TXA6YQSD3GCQQC22GEQ54J2UDCXDXHWN.gfortran-win_amd64.dll
  stacklevel=1)


In [3]:
'''
Globals/System variables
'''

NUM_TOWERS = 5 
RUNS_PER_DATA_POINT = 1
BATCH_SIZES = [5, 10] #how many time steps to parse for tower matches

# batch size stuff -> show how batch size effects optimality
# change to time increments (batch size based on time increment)
# take the sum of the cost vs. batch size, keep synthesis time around (average time per batch)
# should show that as batch size goes down, cost goes up, but synthesis time goes down

#DESTINATIONS = ['K'+ str(i) for i in range(1, NUM_TOWERS+1)] #randomize this?
# DESTINATIONS = dict()
# for i in range(1, NUM_TOWERS+1):
#     DESTINATIONS['K' + str(i)] = []

# DATA indicies
EVENT_TIME_INDEX = 0
ORIGIN_INDEX = 1
DESTINATION_INDEX = 2
DEPARTURE_TIME_INDEX = 3
NUM_PASSENGERS_INDEX = 4


In [4]:
'''
Read NASA data
'''
import csv
import copy

DESTINATIONS = {'K4' : [], 'K6' : [], 'K10' : [], 'K11' : [], 'K14': []}

# print(DESTINATIONS)

data = open('data/scn_UAM_testNewVT.trp', 'r')


reader = csv.reader(data)
next(reader) #skip headers
next(reader) #skip units

# structure = destinations_per_batch_size_per_batch[batch_size_index][batch_index] = destination_dict
destinations_per_batch_size_per_batch = [[copy.deepcopy(DESTINATIONS)] for i in range(len(BATCH_SIZES))]
# print(type(destinations_per_batch_size_per_batch[0]))

# print(destinations_per_batch_size_per_batch)
# time_stamp_per_batch_size = [0 for i in range(len(BATCH_SIZES))]
# previous_time = 21633 # first time in file
request_counter = 0
was_a_request = False
for row in reader:    
#     for index, time_stamp in enumerate(time_stamp_per_batch_size):
#         if time_stamp >= BATCH_SIZES[index]:
#             destinations_per_batch_size_per_batch[index].append(copy.deepcopy(DESTINATIONS))
#             time_stamp_per_batch_size[index] = 0
    # 
    for index, batch_size in enumerate(BATCH_SIZES):
        if was_a_request and request_counter % batch_size == 0:
            destinations_per_batch_size_per_batch[index].append(copy.deepcopy(DESTINATIONS))
    

    eventTime, origin, destination, desiredDepartureTime, numPassengers = row

        
    if destination in DESTINATIONS.keys():
        for index in range(len(destinations_per_batch_size_per_batch)):
            last_item_index = len(destinations_per_batch_size_per_batch[index]) - 1 
            destinations_per_batch_size_per_batch[index][last_item_index][destination].append(desiredDepartureTime)
        request_counter+=1
        was_a_request = True
    else:
        was_a_request = False

print(len(destinations_per_batch_size_per_batch[0]))
print("")
print(len(destinations_per_batch_size_per_batch[1]))
for row in destinations_per_batch_size_per_batch[1]:
    for key in row.keys():
        print(len(row[key]))

70

35
1
9
0
0
0
2
3
3
2
0
0
8
1
0
1
2
3
3
0
2
2
3
3
1
1
2
5
1
1
1
4
4
1
0
1
1
6
1
1
1
5
2
1
2
0
3
4
1
1
1
4
3
3
0
0
2
8
0
0
0
3
3
3
0
1
0
3
2
1
4
1
4
2
1
2
3
4
0
0
3
7
1
1
0
1
7
0
1
0
2
7
2
1
0
0
6
3
0
0
1
5
5
0
0
0
5
4
1
0
0
5
3
0
2
0
6
2
0
1
1
2
6
1
1
0
6
0
0
1
3
5
1
0
2
2
2
1
3
2
2
2
3
0
3
2
2
1
0
3
4
3
0
1
1
5
1
1
4
0
4
0
1
5
1
3
2
2
0
1
5
1
0
1
1
6


In [5]:
'''
Create a system per interval

current choices:
    - requests have no preference
    - one port per tower with 3 capacity
    - request time is equal to index + 1, capped at 5. for 7 requests = [1,2,3,4,5,5,5]
'''


system_per_batch_per_interval = [[] for i in range(len(BATCH_SIZES))]

TIME_CAP = 3
REQ_CAP = 9999
for index, system_per_interval in enumerate(system_per_batch_per_interval):
    print("\n\nnew request batch size " + str(BATCH_SIZES[index]))
    for count, row in enumerate(destinations_per_batch_size_per_batch[index]):
        print("new system " + str(count))
        system = []
        for key in row.keys():
        #graph_manager.return_tower_specific(port_dict, req_per_step, request_vec, time_req)
            port_dict = {'0' : 3}
            req_per_step = 1
            request_vec = []
            for index, i in enumerate(row[key]):
                if index >= REQ_CAP:
                    break
                request_vec.append('no_pref')
            
            time_vec = []
            for i in range(1, len(request_vec)+1):
                if(i > TIME_CAP):
                    time_vec.append(TIME_CAP)
                else:
                    time_vec.append(i)

            assert(len(request_vec) == len(time_vec))
#             print("new tower")
            tower = graph_manager.return_tower_specific(port_dict, req_per_step, request_vec, time_vec)
            system.append(tower)
        assert(len(system) == len(row.keys()))
        system_per_interval.append(system)
        print(system)





new request batch size 5
new system 0
[<reworked_graph.ReworkedGraph object at 0x0000016672DA8C50>, <reworked_graph.ReworkedGraph object at 0x0000016672DA8C88>, <reworked_graph.ReworkedGraph object at 0x0000016672DF4AC8>, <reworked_graph.ReworkedGraph object at 0x0000016672DBC320>, <reworked_graph.ReworkedGraph object at 0x0000016672E05A58>]
new system 1
[<reworked_graph.ReworkedGraph object at 0x0000016672E05278>, <reworked_graph.ReworkedGraph object at 0x0000016672E05240>, <reworked_graph.ReworkedGraph object at 0x0000016672E05AC8>, <reworked_graph.ReworkedGraph object at 0x0000016672E0D6A0>, <reworked_graph.ReworkedGraph object at 0x0000016672E12E10>]
new system 2
[<reworked_graph.ReworkedGraph object at 0x0000016672DD16A0>, <reworked_graph.ReworkedGraph object at 0x0000016672E12BE0>, <reworked_graph.ReworkedGraph object at 0x0000016672E155F8>, <reworked_graph.ReworkedGraph object at 0x0000016672E158D0>, <reworked_graph.ReworkedGraph object at 0x0000016672E1BA20>]
new system 3
[<r

[<reworked_graph.ReworkedGraph object at 0x0000016673106F28>, <reworked_graph.ReworkedGraph object at 0x000001667310BBA8>, <reworked_graph.ReworkedGraph object at 0x000001667310E0F0>, <reworked_graph.ReworkedGraph object at 0x000001667310E2E8>, <reworked_graph.ReworkedGraph object at 0x000001667310E4A8>]
new system 69
[<reworked_graph.ReworkedGraph object at 0x0000016673119F28>, <reworked_graph.ReworkedGraph object at 0x000001667310E6D8>, <reworked_graph.ReworkedGraph object at 0x0000016673114B00>, <reworked_graph.ReworkedGraph object at 0x0000016673114EF0>, <reworked_graph.ReworkedGraph object at 0x0000016673114518>]


new request batch size 10
new system 0
[<reworked_graph.ReworkedGraph object at 0x0000016672F37358>, <reworked_graph.ReworkedGraph object at 0x0000016672F37F98>, <reworked_graph.ReworkedGraph object at 0x0000016673129898>, <reworked_graph.ReworkedGraph object at 0x00000166730C89B0>, <reworked_graph.ReworkedGraph object at 0x00000166731329B0>]
new system 1
[<reworked_gra

[<reworked_graph.ReworkedGraph object at 0x00000166733AB320>, <reworked_graph.ReworkedGraph object at 0x00000166733B6D68>, <reworked_graph.ReworkedGraph object at 0x00000166733AB5F8>, <reworked_graph.ReworkedGraph object at 0x00000166733BAE48>, <reworked_graph.ReworkedGraph object at 0x00000166733BF978>]
new system 26
[<reworked_graph.ReworkedGraph object at 0x00000166733BF6A0>, <reworked_graph.ReworkedGraph object at 0x00000166733BF9B0>, <reworked_graph.ReworkedGraph object at 0x00000166733D0D68>, <reworked_graph.ReworkedGraph object at 0x00000166733D4D30>, <reworked_graph.ReworkedGraph object at 0x00000166733D9748>]
new system 27
[<reworked_graph.ReworkedGraph object at 0x00000166733DF518>, <reworked_graph.ReworkedGraph object at 0x00000166733DFF28>, <reworked_graph.ReworkedGraph object at 0x00000166733E3E48>, <reworked_graph.ReworkedGraph object at 0x00000166733E5588>, <reworked_graph.ReworkedGraph object at 0x00000166733E5748>]
new system 28
[<reworked_graph.ReworkedGraph object at

In [6]:
'''
Sum total cost of batch
'''

costs_per_system_per_batch = []
minimized_cost_per_system_per_batch = []
time_per_system_per_batch = []
num_rounds_per_system_per_batch = []
for index, batch_size in enumerate(BATCH_SIZES):
    costs_per_system_per_batch.append([])
    time_per_system_per_batch.append([])
    num_rounds_per_system_per_batch.append([])
    minimized_cost_per_system_per_batch.append([])
    for system in system_per_batch_per_interval[index]:
        graph_manager.reset_globals()
        total_time, num_rounds, minimized_cost_vec, system_timings, cost_vec_per_round = graph_manager.run_minimizing_mvp(
            copy.deepcopy(system),
            rollout_index=0
        )
        costs_per_system_per_batch[index].append(cost_vec_per_round)
        time_per_system_per_batch[index].append(system_timings)
        num_rounds_per_system_per_batch[index].append(num_rounds)
        minimized_cost_per_system_per_batch[index].append(minimized_cost_vec)
        assert(minimized_cost_vec == cost_vec_per_round[len(cost_vec_per_round)-1])

Tower has requests :: ['no_pref'] and times of [1] and ports of {'0': 3}
Tower has requests :: ['no_pref', 'no_pref', 'no_pref', 'no_pref'] and times of [1, 2, 3, 3] and ports of {'0': 3}
Tower has requests :: [] and times of [] and ports of {'0': 3}
Tower has requests :: [] and times of [] and ports of {'0': 3}
Tower has requests :: [] and times of [] and ports of {'0': 3}
Most expensive request cost: [0, 0, 1]
Most expensive request cost: [0, 2, 1]
Most expensive request cost: [-1, -1, -1]
Most expensive request cost: [-1, -1, -1]
Most expensive request cost: [-1, -1, -1]
accepting tower index 0
lowest_new_cost [0, 0, 2]
cost of accepting request list :: [[0, 0, 2], [0, 0, 2], [0, 0, 2], [0, 0, 2]]

 new round - completed in time 0.07105759999999961
	round 0 breakdown :: 
	 tower 0 took 0.005239999999999689 to find most expensive request and took 0.003923100000000179 to synthesize with published request
	 tower 1 took 0.05045680000000008 to find most expensive request and took 0 to s

Most expensive request cost: [0, 0, 1]
Most expensive request cost: [0, 0, 1]
Most expensive request cost: [0, 0, 1]
Most expensive request cost: [0, 0, 1]
Most expensive request cost: [-1, -1, -1]
accepting tower index -1
lowest_new_cost [0, 0, 1]
cost of accepting request list :: [[0, 0, 2], [0, 0, 2], [0, 0, 2], [0, 0, 2]]
accepting tower index -1
lowest_new_cost [0, 0, 1]
cost of accepting request list :: [[0, 0, 2], [0, 0, 2], [0, 0, 2], [0, 0, 2]]
accepting tower index -1
lowest_new_cost [0, 0, 1]
cost of accepting request list :: [[0, 0, 2], [0, 0, 2], [0, 0, 2], [0, 0, 2]]
accepting tower index -1
lowest_new_cost [0, 0, 1]
cost of accepting request list :: [[0, 0, 2], [0, 0, 2], [0, 0, 2], [0, 0, 2]]
accepting tower index -1
lowest_new_cost [-1, -1, -1]
cost of accepting request list :: [[0, 0, 2], [0, 0, 2], [0, 0, 2], [0, 0, 2]]

 new round - completed in time 0.11494500000000007
	round 0 breakdown :: 
	 tower 0 took 0.00440599999999991 to find most expensive request and took

accepting tower index -1
lowest_new_cost [0, 0, 1]
cost of accepting request list :: [[0, 2, 2], [0, 0, 2], [0, 0, 2], [0, 0, 2]]
accepting tower index -1
lowest_new_cost [0, 0, 1]
cost of accepting request list :: [[0, 0, 2], [0, 0, 2], [0, 0, 2], [0, 0, 2]]
accepting tower index -1
lowest_new_cost [-1, -1, -1]
cost of accepting request list :: [[0, 2, 2], [0, 0, 2], [0, 0, 2], [0, 0, 2]]
accepting tower index -1
lowest_new_cost [-1, -1, -1]
cost of accepting request list :: [[0, 2, 2], [0, 0, 2], [0, 0, 2], [0, 0, 2]]
accepting tower index -1
lowest_new_cost [-1, -1, -1]
cost of accepting request list :: [[0, 2, 2], [0, 0, 2], [0, 0, 2], [0, 0, 2]]

 new round - completed in time 0.14967580000000025
	round 2 breakdown :: 
	 tower 0 took 0.0002380999999997968 to find most expensive request and took 0.013241100000000117 to synthesize with published request
	 tower 1 took 0.007666900000000254 to find most expensive request and took 0.07747210000000004 to synthesize with published reques

accepting tower index -1
lowest_new_cost [-1, -1, -1]
cost of accepting request list :: [[0, 0, 2], [0, 0, 2], [0, 0, 2], [0, 0, 2]]

 new round - completed in time 0.11163169999999978
	round 0 breakdown :: 
	 tower 0 took 0.012519199999999842 to find most expensive request and took 0.02669290000000002 to synthesize with published request
	 tower 1 took 0.0015019999999998923 to find most expensive request and took 0.018356299999999326 to synthesize with published request
	 tower 2 took 5.0999999996470535e-06 to find most expensive request and took 0.01592170000000026 to synthesize with published request
	 tower 3 took 2.700000000022129e-06 to find most expensive request and took 0.018177499999999736 to synthesize with published request
	 tower 4 took 2.9000000001389026e-06 to find most expensive request and took 0.01693110000000031 to synthesize with published request

violation minimized
round breakdown average
tower 0 expensive request cumulative : 0.012519199999999842 ,published req

accepting tower index -1
lowest_new_cost [-1, -1, -1]
cost of accepting request list :: [[0, 0, 2], [0, 0, 2], [0, 0, 2], [0, 0, 2]]

 new round - completed in time 0.14902950000000015
	round 0 breakdown :: 
	 tower 0 took 0.012303600000000081 to find most expensive request and took 0.0260328000000003 to synthesize with published request
	 tower 1 took 0.0015716000000001173 to find most expensive request and took 0.01913609999999899 to synthesize with published request
	 tower 2 took 3.299999999484271e-06 to find most expensive request and took 0.018329999999998847 to synthesize with published request
	 tower 3 took 5.100000000091143e-06 to find most expensive request and took 0.05317509999999803 to synthesize with published request
	 tower 4 took 2.7999999998584713e-06 to find most expensive request and took 0.01666119999999971 to synthesize with published request

violation minimized
round breakdown average
tower 0 expensive request cumulative : 0.012303600000000081 ,published reques

...

accepting tower index -1
lowest_new_cost [0, 0, 1]
cost of accepting request list :: [[0, 2, 2], [0, 0, 2], [0, 0, 2], [0, 2, 2]]
accepting tower index -1
lowest_new_cost [0, 0, 1]
cost of accepting request list :: [[0, 2, 2], [0, 0, 2], [0, 0, 2], [0, 2, 2]]
accepting tower index -1
lowest_new_cost [0, 0, 1]
cost of accepting request list :: [[0, 2, 2], [0, 2, 2], [0, 0, 2], [0, 2, 2]]
accepting tower index -1
lowest_new_cost [0, 0, 1]
cost of accepting request list :: [[0, 2, 2], [0, 2, 2], [0, 0, 2], [0, 0, 2]]
accepting tower index -1
lowest_new_cost [-1, -1, -1]
cost of accepting request list :: [[0, 2, 2], [0, 0, 2], [0, 0, 2], [0, 2, 2]]

 new round - completed in time 0.2914917000000017
	round 3 breakdown :: 
	 tower 0 took 0.007089100000001736 to find most expensive request and took 0.05250849999999829 to synthesize with published request
	 tower 1 took 3.299999999484271e-06 to find most expensive request and took 0.07883720000000238 to synthesize with published request
	 towe

Most expensive request cost: [0, 0, 1]
Most expensive request cost: [0, 0, 1]
Most expensive request cost: [0, 0, 1]
Most expensive request cost: [0, 0, 1]
Most expensive request cost: [0, 0, 1]
accepting tower index -1
lowest_new_cost [0, 0, 1]
cost of accepting request list :: [[0, 0, 2], [0, 2, 2], [0, 0, 2], [0, 0, 2]]
accepting tower index -1
lowest_new_cost [0, 0, 1]
cost of accepting request list :: [[0, 0, 2], [0, 2, 2], [0, 0, 2], [0, 0, 2]]
accepting tower index -1
lowest_new_cost [0, 0, 1]
cost of accepting request list :: [[0, 0, 2], [0, 0, 2], [0, 0, 2], [0, 0, 2]]
accepting tower index -1
lowest_new_cost [0, 0, 1]
cost of accepting request list :: [[0, 0, 2], [0, 0, 2], [0, 2, 2], [0, 0, 2]]
accepting tower index -1
lowest_new_cost [0, 0, 1]
cost of accepting request list :: [[0, 0, 2], [0, 0, 2], [0, 2, 2], [0, 0, 2]]

 new round - completed in time 0.2459910999999977
	round 0 breakdown :: 
	 tower 0 took 0.012324499999998295 to find most expensive request and took 0.036

Tower has requests :: ['no_pref', 'no_pref', 'no_pref'] and times of [1, 2, 3] and ports of {'0': 3}
Tower has requests :: [] and times of [] and ports of {'0': 3}
Tower has requests :: ['no_pref'] and times of [1] and ports of {'0': 3}
Tower has requests :: ['no_pref'] and times of [1] and ports of {'0': 3}
Tower has requests :: ['no_pref', 'no_pref', 'no_pref', 'no_pref', 'no_pref'] and times of [1, 2, 3, 3, 3] and ports of {'0': 3}
Most expensive request cost: [0, 0, 1]
Most expensive request cost: [-1, -1, -1]
Most expensive request cost: [0, 0, 1]
Most expensive request cost: [0, 0, 1]
Most expensive request cost: [2, 2, 1]
accepting tower index 1
lowest_new_cost [0, 0, 2]
cost of accepting request list :: [[0, 2, 2], [0, 0, 2], [0, 0, 2], [0, 0, 2]]

 new round - completed in time 0.1921128000000003
	round 0 breakdown :: 
	 tower 0 took 0.06958360000000141 to find most expensive request and took 0.02001849999999905 to synthesize with published request
	 tower 1 took 0.00171260000

Tower has requests :: [] and times of [] and ports of {'0': 3}
Tower has requests :: ['no_pref'] and times of [1] and ports of {'0': 3}
Tower has requests :: ['no_pref', 'no_pref', 'no_pref', 'no_pref', 'no_pref'] and times of [1, 2, 3, 3, 3] and ports of {'0': 3}
Tower has requests :: ['no_pref'] and times of [1] and ports of {'0': 3}
Tower has requests :: ['no_pref', 'no_pref', 'no_pref'] and times of [1, 2, 3] and ports of {'0': 3}
Most expensive request cost: [-1, -1, -1]
Most expensive request cost: [0, 0, 1]
Most expensive request cost: [2, 2, 1]
Most expensive request cost: [0, 0, 1]
Most expensive request cost: [0, 0, 1]
accepting tower index 0
lowest_new_cost [0, 0, 2]
cost of accepting request list :: [[0, 0, 2], [0, 0, 2], [0, 0, 2], [0, 2, 2]]

 new round - completed in time 0.15307579999999987
	round 0 breakdown :: 
	 tower 0 took 0.0014994999999977665 to find most expensive request and took 0.0029064000000005308 to synthesize with published request
	 tower 1 took 0.002803

accepting tower index -1
lowest_new_cost [-1, -1, -1]
cost of accepting request list :: [[2, 2, 2], [0, 0, 2], [0, 0, 2], [0, 2, 2]]

 new round - completed in time 0.21122890000000183
	round 2 breakdown :: 
	 tower 0 took 3.8999999958377884e-06 to find most expensive request and took 0.040236099999994224 to synthesize with published request
	 tower 1 took 0.005869300000000521 to find most expensive request and took 0.05231870000000072 to synthesize with published request
	 tower 2 took 2.0000000020559128e-06 to find most expensive request and took 0.015550799999985543 to synthesize with published request
	 tower 3 took 1.700000005655511e-06 to find most expensive request and took 0.018802500000006717 to synthesize with published request
	 tower 4 took 0.007363499999996748 to find most expensive request and took 0.06946310000000011 to synthesize with published request

violation minimized
round breakdown average
tower 0 expensive request cumulative : 0.017992299999995964 ,published req

In [7]:
'''
data buffer
'''
cost_buffer = copy.deepcopy(costs_per_system_per_batch)
time_buffer = copy.deepcopy(time_per_system_per_batch)
num_rounds_buffer = copy.deepcopy(num_rounds_per_system_per_batch)


In [8]:
cost_sums = [[0,0] for i in range(len(BATCH_SIZES))]
# print(cost_buffer)
for index, batch_size in enumerate(BATCH_SIZES):
    print(len(cost_buffer[index]))
    for system_cost in cost_buffer[index]:
        for round_index in range(len(system_cost)):
            last_item_index = len(system_cost[round_index])-1
            vec = system_cost[round_index][last_item_index]
            if vec[0] < 0:
                continue
            cost_sums[index][0] += vec[0]
            cost_sums[index][1] += vec[1]
#     for tower_index in range(len(cost_buffer[index][len(cost_buffer[index])-1])):
        
#         cost_sums[index][0] += cost_buffer[index][len(cost_buffer[index])-1][tower_index][0] # level 0
#         cost_sums[index][1] += cost_buffer[index][len(cost_buffer[index])-1][tower_index][1] # level 1
print(cost_sums)

70
35
[[0, 2], [9, 32]]
